In [3]:
pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [12]:
import time
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/STOCK_MARKET_PHASE2/phase_2/codes/")
%run PAPER_WITH_CODE_STOCK.ipynb

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
df_path = "/content/drive/MyDrive/Colab Notebooks/STOCK_MARKET_PHASE2/phase_2/datasets/NIFTY50_15_years_data.xlsx"
df = pd.read_excel(df_path)
# import hyper params
hyper_params_path = "/content/drive/MyDrive/Colab Notebooks/STOCK_MARKET_PHASE2/phase_2/Hyper_Parameters.xlsx"
hyper_params = pd.read_excel(hyper_params_path, "hyper_params")

In [ ]:
df.head()

,Date,High,Low,Open,Close,Volume,Adj Close
0,2007-09-17,4549.049805,4482.850098,4518.450195,4494.649902,0,4494.649902
1,2007-09-18,4551.799805,4481.549805,4494.100098,4546.200195,0,4546.200195
2,2007-09-19,4739.000000,4550.250000,4550.250000,4732.350098,0,4732.350098
3,2007-09-20,4760.850098,4721.149902,4734.850098,4747.549805,0,4747.549805
4,2007-09-21,4855.700195,4733.700195,4752.950195,4837.549805,0,4837.549805


In [ ]:
df.tail()

,Date,High,Low,Open,Close,Volume,Adj Close
3712,2022-11-14,18399.449219,18311.400391,18376.400391,18329.150391,301400,18329.150391
3713,2022-11-15,18427.949219,18282.000000,18362.750000,18403.400391,250900,18403.400391
3714,2022-11-16,18442.150391,18344.150391,18398.250000,18409.650391,219300,18409.650391
3715,2022-11-17,18417.599609,18312.949219,18358.699219,18343.900391,200500,18343.900391
3716,2022-11-18,18394.599609,18209.800781,18382.949219,18307.650391,198800,18307.650391


In [ ]:
hyper_params.iloc[:1,:]

,trainig_data_size,window_size,lstm_input_layer,lstm_middle_layer,dense_layer,epochs,RMSE,Accuracy
0,0.85,30.0,100.0,100.0,500.0,5.0,60.839337,98.896823


In [ ]:
%%time
# train FTSE data
run_df_ftse, models_ftse = run_model_iteration(df, hyper_params.iloc[:1,:])

Trial 1 Complete [00h 00m 13s]
multi_objective: 0.0

Best multi_objective So Far: 0.0
Total elapsed time: 00h 00m 13s
18/18 [==============================] - 3s 8ms/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 352)           498432    
                                                                 
 dropout (Dropout)           (None, 30, 352)           0         
                                                                 
 lstm_1 (LSTM)               (None, 30, 352)           992640    
                                                                 
 lstm_2 (LSTM)               (None, 352)               992640    
                                                                 
 dropout_1 (Dropout)         (None, 352)               0         
                                                                 
 dense (Dense)   

In [18]:
def run_model(df, cols, sequence_length):
  split = (0.85)
  x_train, y_train, x_test, y_test, y_test_ori, record_max, record_min = data_preprocess(df, cols, sequence_length, split)

  # build_train_model(keras_tuner.HyperParameters())
  # build_train_model(x_train, y_train, sequence_length, keras_tuner.HyperParameters(),dense_output, drop_out, epochs, batch_size, input_dim)


  tuner = keras_tuner.RandomSearch(
      hypermodel=build_model,
      objective=["val_accuracy"],
      max_trials=2,
      executions_per_trial=2,
      overwrite=True,
      directory="my_dir",
      project_name="stock",
  )

  tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

  models = tuner.get_best_models(num_models=2)
  model = models[0]
  RMSE, accuracy, de_predicted = validation(model, x_test, y_test, y_test_ori, prediction_len, record_max, record_min)
  return model, y_test_ori, RMSE, accuracy, de_predicted

# tuner.results_summary()



In [ ]:
sequence_len = [20,25,30,40,50,60]
models = []
valids = []
cols = ['Close']
for seq_len in sequence_len:
  model, y_test_ori, RMSE, accuracy, de_predicted = run_model(df, cols, seq_len)
  valid = pd.DataFrame({'Actual':y_test_ori.flatten(), 'Predicted':de_predicted})
  models.append(model)
  valids.append(valid)
  print(f"RMSE:{RMSE}, Acc:{accuracy}")
  plt.figure(figsize=(16,8))
  plt.title(f'Model Run NIFTY50 with 15 Years of Data, RMSE={RMSE}')
  plt.plot(valid)
  plt.xlabel('Date', fontsize=(18))
  plt.ylabel('Close Price', fontsize=(18))
  plt.legend(['Close', 'Pred Close'], loc='lower right')
  plt.show()

Trial 1 Complete [00h 04m 57s]
multi_objective: 0.0

Best multi_objective So Far: 0.0
Total elapsed time: 00h 04m 57s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
448               |160               |units
tanh              |tanh              |activation
True              |True              |dropout
0.0001938         |0.00017956        |lr

Epoch 1/5
99/99 [==============================] - 82s 749ms/step - loss: 0.1331 - val_loss: 0.0679
Epoch 2/5
99/99 [==============================] - 77s 775ms/step - loss: 0.0755 - val_loss: 0.0582
Epoch 3/5
99/99 [==============================] - 75s 762ms/step - loss: 0.0607 - val_loss: 0.0493
Epoch 4/5
99/99 [==============================] - 78s 789ms/step - loss: 0.0485 - val_loss: 0.0442
Epoch 5/5
99/99 [==============================] - 77s 784ms/step - loss: 0.0413 - val_loss: 0.0394
Epoch 1/5
99/99 [==============================] - 86s 790ms/step - loss: 0.1265 - val_loss: 0.0667
Epoch 2/5
84/99 [====

In [ ]:
RMSE, accuracy, de_predicted = validation(model, x_test, y_test, y_test_ori, prediction_len, record_max, record_min)

# print("MSE",MSE)
print("RMSE",RMSE)
print('accuracy',accuracy)
# print('mean_error_percent',mean_error_percent)

valid = pd.DataFrame({'Actual':y_test_ori.flatten(), 'Predicted':de_predicted})



plt.figure(figsize=(16,8))
plt.title(f'Model Run NIFTY50 with 15 Years of Data, RMSE={RMSE}')
plt.plot(valid)
plt.xlabel('Date', fontsize=(18))
plt.ylabel('Close Price', fontsize=(18))
plt.legend(['Close', 'Pred Close'], loc='lower right')
plt.show()